In [16]:
import sys
sys.path.append('/home/daniel/Documents/Uni/MT/poi-prediction/')
import pandas as pd
from src.eval import create_prediction_df, run_predictions, create_prediction_poi_files, compute_overall_metrics, compute_poi_wise_metrics, compute_vert_wise_metrics
from BIDS.POI_plotter import visualize_pois
from BIDS import POI
import numpy as np

In [2]:
def viz_poi(
    paths_dict,
    sub,
    vert,
    poi_type = 'pred',
    poi_list = None,
):
    poi_path = paths_dict[sub, vert][poi_type]
    seg_vert = paths_dict[sub, vert]['seg_vert']
    poi = POI.load(poi_path).copy()
    if poi_list is not None:
        new_centroids = {}
        for v, p, c in poi.items():
            if p in poi_list:
                new_centroids[v, p] = c
        poi.centroids = new_centroids
    
    visualize_pois(poi, seg_vert, vert_idx_list = [vert])

In [3]:
dm_path = '/home/daniel/Documents/Uni/MT/experiment_logs/implants/no-surface/all_pois/freeze/SA-DenseNet-PatchTransformer/version_1/data_module_params.json'
model_path = '/home/daniel/Documents/Uni/MT/experiment_logs/implants/no-surface/all_pois/freeze/SA-DenseNet-PatchTransformer/version_1/checkpoints/sad-pt-epoch=44-fine_mean_distance_masked_val=3.92.ckpt'
val_df = create_prediction_df(dm_path, model_path, joint=False, split = 'val')
test_df = create_prediction_df(dm_path, model_path, joint=False, split = 'test')

In [4]:
poi_paths_val = create_prediction_poi_files(dm_path, model_path, poi_file_ending='test_poi.json', save_path='/home/daniel/prediction_files/', split='val', return_paths = True, project=False)
poi_paths_test = create_prediction_poi_files(dm_path, model_path, poi_file_ending='test_poi.json', save_path='/home/daniel/prediction_files/', split='test', return_paths = True, project=False)

In [5]:
val_df

,subject,vertebra,poi_idx,target,coarse,refined,coarse_proj,refined_proj,coarse_proj_dist,refined_proj_dist,loss_mask,coarse_error,refined_error,coarse_proj_error,refined_proj_error
0,10,18,90,"[80.938, 34.998, 52.938]","[79.99982, 32.00103, 48.007374]","[80.52147, 35.869633, 49.413532]","[82, 36, 50]","[81, 37, 49]",4.895207,1.295272,True,5.845777,3.654465,3.280805,4.418109
1,10,18,91,"[43.009, 36.915, 48.224]","[48.01457, 32.01114, 48.00844]","[44.670425, 36.519104, 47.03314]","[45, 36, 49]","[44, 37, 47]",5.097238,0.825730,True,7.010710,2.082119,2.324541,1.577175
2,10,18,94,"[75.278, 57.099, 57.878]","[79.999985, 49.48076, 63.99968]","[74.85082, 48.298893, 59.473824]","[79, 50, 63]","[72, 49, 61]",1.506300,3.308769,True,10.854018,8.953826,9.512306,9.278252
3,10,18,95,"[49.952, 57.145, 55.651]","[48.138145, 58.12639, 60.103874]","[50.849228, 57.845085, 56.47449]","[47, 58, 61]","[54, 56, 55]",1.454094,3.937744,True,4.907267,1.404732,6.169045,4.256893
4,10,19,90,"[82.789, 37.002, 46.912]","[80.0, 32.06723, 48.062702]","[82.14875, 35.762863, 47.750294]","[81, 34, 50]","[82, 36, 47]",2.913541,0.800813,True,5.783994,1.627304,4.663505,1.278385
5,10,19,91,"[42.522, 38.11, 46.039]","[48.001633, 34.906845, 48.007683]","[43.616997, 36.909786, 46.87338]","[46, 35, 48]","[43, 37, 47]",2.003814,0.636283,True,6.645472,1.826395,5.061038,1.544054
6,10,19,94,"[78.904, 57.113, 56.73]","[79.999985, 55.83317, 63.9988]","[79.605545, 54.42607, 58.01788]","[80, 54, 64]","[84, 54, 57]",1.833172,4.530877,True,7.461540,3.061108,7.984039,5.977699
7,10,19,95,"[47.846, 58.002, 55.085]","[48.059563, 60.518906, 60.693024]","[47.977158, 59.03744, 57.654655]","[49, 60, 65]","[52, 55, 56]",4.438889,5.934819,True,6.150640,2.773531,10.179930,5.206240
8,10,20,90,"[85.241, 39.149002, 46.837]","[80.00001, 33.827347, 48.044067]","[81.94596, 37.122505, 48.356983]","[82, 36, 51]","[83, 38, 49]",4.178268,1.514750,True,7.566042,4.156236,6.144171,3.319764
9,10,20,91,"[41.627, 39.145, 46.911]","[48.000214, 47.986065, 48.00067]","[43.506924, 46.79895, 49.509727]","[48, 47, 48]","[46, 45, 48]",0.986065,3.425044,True,10.953067,8.298822,10.173597,7.388510


In [7]:
viz_poi(paths_dict=poi_paths_val, sub='10', vert=19)

[ ] Image reoriented from ('L', 'A', 'S') to ('P', 'I', 'R')
[*] Centroids reoriented from ('L', 'A', 'S') to ('P', 'I', 'R')
('P', 'I', 'R') ('P', 'I', 'R')


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 75.29it/s]


Widget(value="<iframe src='http://localhost:44143/index.html?ui=P_0x7b50f31ddf30_1&reconnect=auto' style='widt…

In [29]:
def get_angles(poi_paths, k):
    sub, vert = k
    poi_pred = POI.load(poi_paths[sub, vert]['pred'])
    poi_gt = POI.load(poi_paths[sub, vert]['gt'])

    s94_90_pred = np.array(poi_pred[vert, 94]) - np.array(poi_pred[vert, 90])
    s94_90_gt = np.array(poi_gt[vert, 94]) - np.array(poi_gt[vert, 90])

    s_95_91_pred = np.array(poi_pred[vert, 95]) - np.array(poi_pred[vert, 91])
    s_95_91_gt = np.array(poi_gt[vert, 95]) - np.array(poi_gt[vert, 91])

    theta94_90 = np.arccos(np.dot(s94_90_pred, s94_90_gt) / (np.linalg.norm(s94_90_pred) * np.linalg.norm(s94_90_gt))) * 180 / np.pi
    theta95_91 = np.arccos(np.dot(s_95_91_pred, s_95_91_gt) / (np.linalg.norm(s_95_91_pred) * np.linalg.norm(s_95_91_gt))) * 180 / np.pi

    return (theta94_90, theta95_91)

In [35]:
all_angles_val = []
for k in poi_paths_val.keys():
    for i in range(3):
        all_angles_val.append(get_angles(poi_paths_val, k))

all_angles = np.array(all_angles_val)
all_angles.mean(), all_angles.std()

(9.136150279778473, 6.187779133847998)

In [36]:
all_angles_test = []
for k in poi_paths_test.keys():
    for i in range(3):
        all_angles_test.append(get_angles(poi_paths_test, k))

all_angles = np.array(all_angles_test)
all_angles.mean(), all_angles.std()

(11.940403955062774, 5.806195687445854)

In [38]:
#Add a column split to the dataframes then concatenate them
val_metrics = compute_overall_metrics(val_df)
val_metrics['Split'] = 'Val'
test_metrics = compute_overall_metrics(test_df)
test_metrics['Split'] = 'Test'

metrics = pd.concat([val_metrics, test_metrics])
print(metrics.to_latex())

\begin{tabular}{rrrrrl}
\toprule
Mean Error & Median Error & MSE & Accuracy & Max Error & Split \\
\midrule
7.653461 & 7.492056 & 62.675667 & 0.000000 & 11.023216 & Val \\
3.904238 & 3.381812 & 19.476263 & 0.200000 & 8.953826 & Val \\
7.627388 & 8.145839 & 65.297830 & 0.000000 & 11.441132 & Val \\
4.533947 & 4.576163 & 24.443122 & 0.150000 & 9.278252 & Val \\
7.249942 & 6.505943 & 59.534962 & 0.000000 & 12.683422 & Test \\
5.432266 & 4.844140 & 35.058346 & 0.000000 & 10.029899 & Test \\
7.399155 & 6.765236 & 61.857048 & 0.000000 & 12.687757 & Test \\
5.734457 & 6.268641 & 38.061144 & 0.100000 & 9.088847 & Test \\
\bottomrule
\end{tabular}



In [39]:
#For pretty printing, bring split column to the front
cols = metrics.columns.tolist()
cols = cols[-1:] + cols[:-1]
metrics = metrics[cols]

#Make the index column a regular column called 'Prediction Type'
metrics.reset_index(inplace=True)
metrics.rename(columns={'index': 'Prediction Type'}, inplace=True)
#Rename the entries in the 'Prediction Type' column coarse_error -> Coarse, refined_error -> Fine, coarse_proj_error -> Coarse with projection, refined_proj_error -> Fine with projection
metrics['Prediction Type'] = metrics['Prediction Type'].replace({'coarse_error': 'Coarse', 'refined_error': 'Fine', 'coarse_proj_error': 'Coarse with projection', 'refined_proj_error': 'Fine with projection'})

# Reorder the columns for pretty printing
metrics = metrics[['Split', 'Prediction Type', 'Mean Error', 'Median Error', 'MSE', 'Accuracy', 'Max Error']]
#drop the max error
metrics = metrics.drop(columns=['Max Error'])
print(metrics.to_latex(index=False, float_format='%.2f'))

\begin{tabular}{llrrrr}
\toprule
Split & Prediction Type & Mean Error & Median Error & MSE & Accuracy \\
\midrule
Val & Coarse & 7.65 & 7.49 & 62.68 & 0.00 \\
Val & Fine & 3.90 & 3.38 & 19.48 & 0.20 \\
Val & Coarse with projection & 7.63 & 8.15 & 65.30 & 0.00 \\
Val & Fine with projection & 4.53 & 4.58 & 24.44 & 0.15 \\
Test & Coarse & 7.25 & 6.51 & 59.53 & 0.00 \\
Test & Fine & 5.43 & 4.84 & 35.06 & 0.00 \\
Test & Coarse with projection & 7.40 & 6.77 & 61.86 & 0.00 \\
Test & Fine with projection & 5.73 & 6.27 & 38.06 & 0.10 \\
\bottomrule
\end{tabular}

